In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [40]:
def depend_extract(t): 
    doc = nlp(t)
    for sent in doc.sents:
        for token in sent:
            # if is_declinable(token):
            if token.pos_ == "VERB":
                    # print()
                    # register_pred_count(pred_cound_dict, token.lemma_)
                for c in token.children:
                        # print(c)
                    for c2 in c.children:
                        if c2.dep_ == "case" and c2.orth_ in case_essential:
        
                            print( c.lemma_, c2, token.lemma_)

In [29]:
t = "インターネットは排出量が多い"
t = "マドリッドの原宿、表参道エリアを中心に人気専門店が続々登場するなど、今年一躍話題になったポップコーンですが、日本穀物検定協会が金曜ロードSHOW!とコラボレーションしたきょうは会社休みます。を12月に発売するそうです。"
# t = "固有抽出表現は、文から日付や地名等の固有表現を抽出する処理です。"
# t = "天気が外に出ますよね"
t  = "選手と後一人は誰でしたっけ？"

In [30]:
from spacy import displacy
displacy.render(nlp(t), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [41]:
depend_extract(t)

In [37]:
def is_declinable(token):
    pos = token.tag_.split("-")
    try:
        if pos[0] == "動詞" or pos[0] == "形容詞" or pos[2] == "サ変可能":
            # print(token.lemma_)
            return True
        else:
            False
    except:
        return False

In [38]:
case_essential = set("が を に で".split())


def RDF_test(text):
    text = clean_text(text)
    doc = nlp(text)
    phrases = ""
    for i, token in enumerate( doc ):
        # phrases.append("")
        # if token.pos_ in ["VERB", "ADJ", "AUX"]:
        if is_declinable(token):
            # print(token.lemma_)
            phrase = ""
            for c in token.children:
                if "名詞" in c.tag_:
                    # print("\t", c.lemma_, c.dep_)
                    # noun = extract_continue_noun(c)
                    noun = c.orth_
                    # print("\t", extract_continue_noun(c), c.dep_)
                    # print("\t", extract_continue_noun(c))
                    count_case = 0
                    for c2 in c.children:
                        if c2.dep_ == "case" and c2.orth_ in case_essential:
                            count_case += 1
                            # print("\t", c, c2, token.lemma_)
                            if count_case >= 2:
                                phrase = phrase + c2.orth_
                            else:
                                phrase = phrase + noun+ c2.orth_
                            
            # トリプルがない
            if phrase=="":
                continue
            phrase += token.lemma_
            # 形容詞 
            if (token.pos_=="ADJ" or "名詞" in token.tag_) and i<len(doc)-1:
                phrase += doc[i+1].lemma_
            
            # 連結
            if phrases=="":
                phrases = phrase
            else:
                phrases = phrases+"。"+phrase

            # print(phrase)
    return phrases

In [36]:
RDF_test(t)

''

In [19]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [20]:
error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [42]:
rdf_list = []
for utt in sys_utt[500:]:
    print(utt)
    # rdf_list.append()
    print(">>>")
    # print(RDF_test(utt))
    depend_extract(utt)
    
    print("-------------")

世界大会と言えば、2020年に蔵王ハートランドでGet Your Guideが開催されることが決定しましたね。
>>>
年 に 開催
ハートランド で 開催
GUIDE が 開催
こと が 決定
-------------
中田英寿さんは知的なサッカーをされる方だったので、個人的には中田英寿さんと柴咲コウさんの組み合わせがとてもお似合いだと思っていたのです。
>>>
サッカー を する
-------------
パルマの奥様は魔女というお菓子は、季節によっていろいろな味が展開されているので、店頭で見かけるとつい買ってしまいます。
>>>
季節 に 展開
味 が 展開
店頭 で 見かける
-------------
エボラウイルスがHNKに入ってくるのを、できるかぎり防いでほしいものです。
>>>
ウイルス が 入る
hnk に 入る
くる を 防ぐ
-------------
大学にはHNK全国から学生が集まり、環境も違いますからウィルスが蔓延しやすいですよね。
>>>
大学 に 集まる
学生 が 集まる
-------------
HNK在住の海外の人や海外に住むHNK人にフォーカスをあてたバラエティ番組は、最近よく見かけますね。
>>>
海外 に 住む
人 に あてる
フォーカス を あてる
-------------
ゴーストバスターズは世界の辺境に様々な理由を抱えて移住したHNK人を取材するドキュメンタリー番組です。
>>>
辺境 に 抱える
理由 を 抱える
人 を 取材
-------------
私が他に好きな有名人は、HNKご出身の、俳優山田洋二さんです。
>>>
-------------
HNKが舞台になった山田洋二監督の作品は、マシュマロホットチョコレートの他に、1980年に公開となった金曜ロードSHOW!という作品があります。
>>>
hnk が なる
舞台 に なる
年 に なる
作品 が ある
-------------
時々見かけるフジテレビのスペシャルドラマ痛快！明石家電視台シリーズで女優ざわちんさんが演じている板野友美の役もハマっていて面白いと思いました。
>>>
シリーズ で 演ずる
さん が 演ずる
-------------
フジテレビの番組であるBISTRO SMAPの世界行ってみたらホントはこんなトコだった！？のコーナー